# Web Scraping Homework - Mission to Mars
## Terrence Cummings

In [48]:
import pandas as pd
from bs4 import BeautifulSoup
import json
import requests
import pymongo
from splinter import Browser
from selenium import webdriver
import time

# Step 1 - Scraping
## NASA Mars New

Retrieve the following data from a search of the latest news articles at the NASA Mars News website URL:

1. Publication Date

2. Title

3. Summary paragraph



In [49]:
#Because the search results at the URL are from Javascript use Selenium to scrape the data

#URL for NASA Mars News website. This show 40 articles from a search of the criteria "Latest" and "All Categories".
#Results of the search are generated by Javascript so not viewable in the webpage HTML
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

#Initialize lists to store Selenium objects
dates = []
titles = []
summarys = []

#Use Selenium to get the needed fields from the JS results
#XPath for tags were found by right-clicking on the tag in the Chrome Inspector tool the Copy XPath
driver = webdriver.Chrome()
driver.get(url)

#Add a delay to give the scraper time to acquire the data
time.sleep(10)
dates = driver.find_elements_by_xpath('//*[@id="page"]/div[3]/div/article/div/section/div/ul/li[*]/div/div/div[1]')
titles = driver.find_elements_by_xpath('//*[@id="page"]/div[3]/div/article/div/section/div/ul/li[*]/div/div/div[2]/a')
summarys = driver.find_elements_by_xpath('//*[@id="page"]/div[3]/div/article/div/section/div/ul/li[*]/div/div/div[3]')

# create empty array to store text data extracted from Selenium objects
date_lst = []
title_lst = []
summary_lst = []

# loop over results and extract text from Selenium objects, add to each list
for date in dates:
    article_date = date.text
    date_lst.append(article_date)
for title in titles:
    article_title = title.text
    title_lst.append(article_title)
for summary in summarys:
    article_summary = summary.text
    summary_lst.append(article_summary)

#Make dataframe of NASA Mars Latest News Articles
nasa_mars_articles_df = pd.DataFrame(list(zip(date_lst, title_lst, summary_lst)), columns =['Date', 'Title', 'Summary'])

driver.quit()

In [50]:
#Confirm results of the scraping
nasa_mars_articles_df

,Date,Title,Summary
0,"May 21, 2020",Air Deliveries Bring NASA's Perseverance Mars ...,A NASA Wallops Flight Facility cargo plane tra...
1,"May 19, 2020","NASA Wins 4 Webbys, 4 People's Voice Awards","Winners include the JPL-managed ""Send Your Nam..."
2,"May 18, 2020",NASA's Perseverance Rover Goes Through Trials ...,The agency's new Mars rover is put through a s...
3,"May 7, 2020",NASA's Perseverance Rover Mission Getting in S...,Stacking spacecraft components on top of each ...
4,"May 6, 2020",NASA Perseverance Mars Rover Scientists Train ...,Team members searched for signs of ancient mic...
5,"May 1, 2020",NASA's Perseverance Rover Will Look at Mars Th...,A pair of zoomable cameras will help scientist...
6,"April 30, 2020",Meet the People Behind NASA's Perseverance Rover,These are the scientists and engineers who bui...
7,"April 29, 2020","Q&A with the Student Who Named Ingenuity, NASA...","As a longtime fan of space exploration, Vaneez..."
8,"April 29, 2020",Alabama High School Student Names NASA's Mars ...,Vaneeza Rupani's essay was chosen as the name ...
9,"April 21, 2020",How NASA's Perseverance Mars Team Adjusted to ...,"Like much of the rest of the world, the Mars r..."
